### Importing Libraries

In [32]:
import mysql.connector
import pandas as pd
import simple_colors
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from matplotlib.colors import ListedColormap, BoundaryNorm
import calendar

host = "139.59.34.149"
user = "neemdb"
password = "(#&pxJ&p7JvhA7<B"
database = "cabh_iaq_db"

id_to_fetch = 1202240012

### Connecting with Database

In [33]:
conn = None

try:
    # Connect to the database
    conn = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    print("Database connection successful.")

    # Create a cursor object to interact with the database
    cursor = conn.cursor()

    query = """
    SELECT id, deviceID, datetime, pm25, pm10, aqi, co2, voc, temp, humidity, battery, viral_index
    FROM reading_db
    WHERE deviceID = %s;
    """
    
    # Execute the query, passing the id_to_fetch as a parameter to avoid SQL injection
    cursor.execute(query, (id_to_fetch,))

    # Fetch all rows from the result of the query
    rows = cursor.fetchall()

    # Check if any rows were returned
    if rows:
        # Convert the fetched data into a pandas DataFrame
        df = pd.DataFrame(rows, columns=["id", "deviceID", "datetime", "pm25", "pm10", "aqi", "co2", "voc", "temp", "humidity", "battery", "viral_index"])

        # Convert 'datetime' column to proper datetime format (optional)
        df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M', errors='coerce')

except mysql.connector.Error as e:
    print("Error while connecting to the database or fetching data:", e)

finally:
    if conn:
        cursor.close()  # Close the cursor
        conn.close()    # Close the connection
        print("Connection closed.")


Database connection successful.
Connection closed.


In [34]:
df.head()

,id,deviceID,datetime,pm25,pm10,aqi,co2,voc,temp,humidity,battery,viral_index
0,965151,1202240012,2024-04-19 18:11:11,18.0,57.0,57.0,398.0,169.0,36.8,26.0,5.0,79.0
1,965167,1202240012,2024-04-30 11:08:12,26.0,74.0,74.0,412.0,57.0,33.9,26.0,5.0,78.0
2,965182,1202240012,2024-04-30 11:09:12,27.0,81.0,81.0,412.0,52.0,33.9,26.0,5.0,78.0
3,965198,1202240012,2024-04-30 11:10:12,29.0,81.0,81.0,414.0,60.0,33.9,26.0,5.0,78.0
4,965214,1202240012,2024-04-30 11:11:13,27.0,81.0,81.0,415.0,64.0,33.9,26.0,5.0,78.0


In [35]:
df['datetime']

0        2024-04-19 18:11:11
1        2024-04-30 11:08:12
2        2024-04-30 11:09:12
3        2024-04-30 11:10:12
4        2024-04-30 11:11:13
                 ...        
224941   2024-11-28 17:08:04
224942   2024-11-28 17:09:04
224943   2024-11-28 17:10:05
224944   2024-11-28 17:11:04
224945   2024-11-28 17:12:05
Name: datetime, Length: 224946, dtype: datetime64[ns]

In [36]:
df['datetime'] = pd.to_datetime(df['datetime'], format='%d-%m-%Y %H:%M', errors='coerce')
df.set_index('datetime', inplace=True)

### Creating Heatmaps

In [42]:
def plot_and_save_feature_heatmaps(df, year, start_date, end_date, features, save_dir):
    '''Function for ploting the heatmaps and save it in Particular directory.
    First Creating the directory which is assign at the time of function calling.
    Assigning the criteria for dates.
    Providing threshold values and color bar for particular threshold.

    
    '''
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Convert start_date and end_date to datetime objects to extract months
    start_date = pd.to_datetime(start_date, format='%d-%m-%Y')
    end_date = pd.to_datetime(end_date, format='%d-%m-%Y')
    
    # Get the months between start_date and end_date
    months = pd.date_range(start=start_date, end=end_date, freq='MS').month
    print(f"Months to process: {months}")
    
    unique_ids = df['deviceID'].unique()

    for system_id in unique_ids:
        system_df = df[df['deviceID'] == system_id]
        
        # Create a directory for the current system ID
        system_dir = os.path.join(save_dir, str(system_id))
        if not os.path.exists(system_dir):
            os.makedirs(system_dir)

        # Loop through each feature
        for feature in features:
            indoor_feature = feature  

            # Initialize the figure for each row of subplots (side-by-side heatmaps for the months + color bar)
            fig, axes = plt.subplots(1, len(months) + 1, figsize=(20, 6), 
                                     gridspec_kw={"width_ratios": [1] * len(months) + [0.1]},  # Extra space for color bar
                                     constrained_layout=True)

            # Define custom color map
            color_list = ['#006400', '#228B22', '#FFFF00', '#FF7F00', '#FF0000', '#8B0000']  
            cmap = ListedColormap(color_list)

            feature_boundaries = {
                'aqi': [0, 50, 100, 150, 200, 300, 500],  
                'pm25': [0, 12, 35, 55, 150, 250, 500],  
                'pm10': [0, 20, 50, 100, 250, 350, 500],  
                'co2': [0, 900, 10000],  
                'voc': [0, 500, 1000]  
            }
            
            feature_labels = {
                'aqi': ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe'],
                'pm25': ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe'],
                'pm10': ['Good', 'Satisfactory', 'Moderate', 'Poor', 'Very Poor', 'Severe'],
                'co2': ['Good', 'Poor'],  
                'voc': ['Good', 'Poor']  
            }

            boundaries = feature_boundaries[feature]
            labels = feature_labels[feature]

            # Adjust boundaries to avoid extra tick issue
            if len(boundaries) - 1 != len(labels):
                labels = labels[:-1]  

            calendar_data_list = []  # Store the calendar data for each month to create a common color bar

            # Loop over the months (from start_date to end_date) and plot heatmaps
            for j, m in enumerate(months):
                month_data = system_df[(system_df.index.year == year) & (system_df.index.month == m)][feature]
                num_days = calendar.monthrange(year, m)[1]

                # Adjust the calendar_data size to accommodate 5 weeks (rows)
                calendar_data = np.full((5, 7), np.nan)  # 5 rows to accommodate up to 5 weeks

                for day in range(1, num_days + 1):
                    day_values = month_data[month_data.index.day == day]
                    if not day_values.empty:
                        daily_avg = day_values.mean()

                        # Calculate the position in the calendar grid
                        first_day_of_month = calendar.monthrange(year, m)[0]  # First weekday of the month
                        week_row = (day + first_day_of_month - 1) // 7
                        week_col = (day + first_day_of_month - 1) % 7

                        # Ensure the week_row is within bounds (maximum of 5 rows)
                        if week_row < 5:
                            calendar_data[week_row, week_col] = daily_avg

                calendar_data_list.append(calendar_data)  # Store calendar data for each month

                # Plot the heatmap for the current month on the corresponding subplot
                norm = BoundaryNorm(boundaries, cmap.N)
                sns.heatmap(calendar_data, annot=True, fmt=".0f", cmap=cmap, norm=norm,
                            cbar=False,  
                            xticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'], 
                            yticklabels=[1, 2, 3, 4, 5],  # Update to reflect 5 weeks max
                            ax=axes[j], linewidths=1, linecolor='black')  

                axes[j].set_title(f"{calendar.month_name[m]} {year} - {indoor_feature}", fontsize=14)
                axes[j].set_xlabel("Day of the Week", fontsize=12)
                axes[j].set_ylabel("Week", fontsize=12)
                
            # Create the color bar in the last subplot (empty column for the color bar)
            cbar_ax = axes[-1]  # Access the last axis reserved for the color bar
            norm = BoundaryNorm(boundaries, cmap.N)
            cbar = fig.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cbar_ax, orientation='vertical')

            # Set color bar ticks and labels
            cbar.set_ticks([(b + b_next) / 2 for b, b_next in zip(boundaries[:-1], boundaries[1:])])
            cbar.set_ticklabels(labels)
            cbar.ax.tick_params(labelsize=12)  # Adjust font size of color bar ticks
            cbar.ax.set_ylabel(f"{feature} Levels", fontsize=14)  # Label for the color bar

            # Save the figure as an image for this feature and system ID
            save_path = os.path.join(system_dir, f"{indoor_feature}_Heatmaps.png")
            fig.savefig(save_path, dpi=300)

            plt.close(fig)
            
print(simple_colors.red(f"System ID: {id_to_fetch}",'bold'))
save_directory = r"C:\Users\abhis\System_Trends"  
plot_and_save_feature_heatmaps(df, 2024, '01-09-2024', '26-11-2024', ['aqi', 'pm25', 'pm10', 'co2','voc'], save_directory)
print(simple_colors.green('Done Successfully', 'bold'))  


System ID: 1202240012
Months to process: Index([9, 10, 11], dtype='int32')
Done Successfully
